<a href="https://colab.research.google.com/github/OWEN-JUN/keras_/blob/master/day0810_colab_cifar10_300.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers import *
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, train_test_split


import numpy as np
import os
import tensorflow as tf


data_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.08,
    height_shift_range=0.08,
    horizontal_flip=True
)



(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()



X_train, _, Y_train, _ = train_test_split(X_train, Y_train, test_size=0.994)

X_train = X_train.reshape(X_train.shape[0], 32 ,32, 3)  
X_test = X_test.reshape(X_test.shape[0], 32 ,32, 3)


Y_train = np_utils.to_categorical(Y_train)      
Y_test = np_utils.to_categorical(Y_test)        


IMG_CHANNERLS = 3
IMG_ROWS = 32
IMG_COLS = 32




# print(X_train[0])
# data_scaling(X_train)
# print(X_train[0])
# X_train = X_train.reshape(X_train.shape[0], 32 ,32 ,3).astype('float32') / 255
# print(X_train[0])

def build_network_cnn(keep_prob=0.1, optimizer='adam', conv1 = 20, conv2=30):
    
    inputs = Input(shape=(32,32,3), name='input')
    
    x = Conv2D(conv1,(3,3),padding="same",activation="relu", input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNERLS))(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(conv1, (3,3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=2)(x)
    x = Dropout(keep_prob)(x)
    
    x = Conv2D(conv2,(3,3),padding="same",activation="relu")(x)
    x = BatchNormalization()(x)
    x = Conv2D(conv2, (3,3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=2)(x)
    x = Dropout(keep_prob)(x)
    
    
#     x = Conv2D(conv3,(3,3),padding="same",activation="relu")(x)
#     x = BatchNormalization()(x)
#     x = Conv2D(conv3, (3,3), activation='relu')(x)
#     x = BatchNormalization()(x)
#     x = MaxPooling2D(pool_size=2)(x)
#     x = Dropout(keep_prob)(x)

    
    
    
    
    
    x = Flatten()(x)
    prediction = Dense(10, activation='softmax', name='output')(x)
    
    model = Model(inputs=inputs, outputs=prediction)
 

    model.compile(loss='categorical_crossentropy',  
                optimizer=optimizer,
                metrics=['accuracy'])
    model.fit_generator(data_generator.flow(X_train, Y_train, batch_size=256),
                        steps_per_epoch=100,     
                        epochs=3,
                        verbose=1)  
    
    return model


def create_hyperparameters():
    batches = [10,20,30,40,50]
    optimizers = ['rmsprop', 'adam', 'adadelta']
    dropout = np.linspace(0.05,0.5, 10)
    epochs = [10,50,100,150, 200]
    conv1 = [30,40,50,60,70]
    conv2 = [30,40,50,60,70]
#     conv3 = [30,40,50,60,70]
    return {"model__batch_size":batches, "model__optimizer":optimizers, "model__keep_prob":dropout, "model__epochs":epochs,"model__conv1":conv1,"model__conv2":conv2}      # Map 형태로 반환

def data_scaling(x_data):
    x_data = x_data.reshape(x_data.shape[0], 32* 32* 3) 
    MinMaxScaler()
    x_data = x_data.reshape(x_data.shape[0], 32, 32, 3)  



Using TensorFlow backend.


170500096/170498071 [==============================] - 9s 0us/step


In [0]:

model = KerasClassifier(build_fn=build_network_cnn, verbose=2)
hyperparameters = create_hyperparameters()


pipe = Pipeline([('minmax', data_scaling(X_train)), ('model', model)])

search = RandomizedSearchCV(estimator=pipe, param_distributions=hyperparameters,
                            n_iter=5,  cv=5)



search.fit(X_train, Y_train)

print("score :", search.best_params_)
print("score :", search.best_estimator_)
print("score :", search.score(X_test, Y_test))